In [1]:
# takes 60s per 40 api calls.
# download player tournament history. skip if already downloaded. player_id list is from BridgePowerRatings.

# previous steps:
# none

# next steps:
# create sql db of player tournament data.

# todo:
# Improve the method of issuing api calls only when needed. Write update info to file?
# Identify the correct player_id from list for each elite player name. Issue is that multiple players can have same name.

In [2]:
import pandas as pd
import pathlib
from collections import defaultdict
import mlBridgeLib

In [3]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [4]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')
acblPath.mkdir(parents=True,exist_ok=True)
bprPath = rootPath.joinpath('bpr')
bprPath.mkdir(parents=True,exist_ok=True)

In [5]:
# takes 60s per 40 api calls.
# call acbl api to retrieve player's tournament data.
# todo: inexplicable variance between 'total' in json and sessions_count. Why? varaince is 0 to -15-ish.
import requests
import urllib
import time
import json
def download_tournaments(df,bearer_file):
    headers = {'accept':'application/json', 'Authorization':bearer[len('Authorization: '):]}
    start_time = time.time()
    get_count = 0
    for n,r in df.iterrows():
        player_ids = r['player_id']
        assert type(player_ids) is list, [type(player_ids), player_ids]
        #print(f"{time.strftime('%X')} player_id list:{player_ids} Name:{r['Name']} Rank:{r['Rank']}")
        print(f"{time.strftime('%X')} Name:{r['Name']} player_id list:{player_ids}")
        for player_id in player_ids:
            if player_id.startswith('tmp:') or player_id.startswith('#'): # somehow #139 crept into player_id
                print(f'Skipping player_id:{player_id}')
                continue
            else:
                print(f'Processing player_id:{player_id}')
            dirPath = acblPath.joinpath('players/'+player_id+'/tournaments')
            if dirPath.exists():
                session_file_count = len(list(dirPath.glob('*.session.json')))
                print(f'dir exists: file count:{session_file_count} dir:{dirPath}')
                if session_file_count > 0: # todo: temp?
                    print(f'dir not empty -- skipping')
                    continue
            else:
                print(f'Creating dir:{dirPath}')
                dirPath.mkdir(parents=True,exist_ok=True)
                session_file_count = 0
            path = 'https://api.acbl.org/v1/tournament/player/history_query'
            query = {'acbl_number':player_id,'page':1,'page_size':50,'start_date':'1900-01-01'}
            params = urllib.parse.urlencode(query)
            url = path+'?'+params
            sessions_count = 0
            while url:
                get_count += 1
                print(f"{n}/{len(df)} gets:{get_count} rate:{round((time.time()-start_time)/get_count,2)} url:{url}")
                #time.sleep(1) # throttle api calling. Maybe not needed as api is taking longer than 1s.
                response = requests.get(url, headers=headers)
                if response.status_code == 500: # unknown response code. try skipping player
                    print(f'500: count:{get_count} skipping') # 4476921 - Thx Merle.
                    next_page_url = None
                    break
                assert response.status_code == 200, (n, url, response.status_code)
                json_response = response.json()
                json_pretty = json.dumps(json_response, indent=4)
                #print(json_pretty)
                next_page_url = json_response['next_page_url']
                sessions_total = json_response['total'] # is same for every page
                if sessions_total == session_file_count: # sometimes won't agree because identical sessions. older scores?
                    print(f'File count correct: {dirPath}: terminating {player_id} early.')
                    next_page_url = None
                    sessions_count = sessions_total
                    break
                for data in json_response['data']:
                    sessions_count += 1 # todo: oops, starts first one at 2. need to move
                    session_id = data['session_id']
                    filePath = dirPath.joinpath(session_id+'.session.json')
                    if filePath.exists():
                        print(f'{sessions_count}/{sessions_total}: File exists: {filePath}: skipping')
                        continue
                    print(f'{sessions_count}/{sessions_total}: Writing:',filePath)
                    with open(filePath,'w',encoding='UTF8') as f:
                        f.write(json.dumps(data, indent=4))
                url = next_page_url
            if sessions_count != sessions_total:
                print(f'Session count mismatch: {dirPath}: variance:{sessions_count-sessions_total}')

In [6]:
# read file containing acbl api bearer secret.
bearer_file = pathlib.Path('c:/sw/bridge/BridgePowerRatings/acbl_api_authorization_bearer_secret.txt')
with open(bearer_file,'rb') as f:
    bearer = f.read()

In [ ]:
# read dfs of BridgePowerRatings. They contain player_id of important players. Download their tournaments history.
files = ['allmps.pkl', 'allpr.pkl', 'd9rank.pkl', 'improv12.pkl', 'improv24.pkl', 'improv6.pkl', 'underate.pkl', 'urpairs.pkl']
files = ['improv24.pkl', 'improv6.pkl', 'underate.pkl', 'urpairs.pkl', 'improv12.pkl']
for f in files:
    df = pd.read_pickle(bprPath.joinpath(f))
    display(f'Processing file:{f}',df.head())
    if 'player_id' in df:
        assert 'Name' in df
        # todo: sort df (e.g. Rank) to better track progress of downloads?
        #df = df['Rank'].astype('float32').sort_values().index
        download_tournaments(df,bearer_file)

'Processing file:improv24.pkl'

,Rank,last/PR,old/PR,PR/inc,Unit,Name,player_id
0,1,63.74,57.30,6.45,503,"Kemper, Carol",[4331699]
1,2,68.27,61.88,6.39,119,"Ayyagari, Murthy",[9146962]
2,3,55.95,49.73,6.21,196,"Drabek, Yan",[5172713]
3,4,60.36,54.20,6.17,425,"Schmahl, Michael",[4934148]
4,5,66.46,60.92,5.54,132,"Kopper, David",[2212501]


11:24:41 Name:Kemper, Carol player_id list:['4331699']
Processing player_id:4331699
dir exists: file count:333 dir:e:\bridge\data\acbl\players\4331699\tournaments
dir not empty -- skipping
11:24:41 Name:Ayyagari, Murthy player_id list:['9146962']
Processing player_id:9146962
dir exists: file count:46 dir:e:\bridge\data\acbl\players\9146962\tournaments
dir not empty -- skipping
11:24:41 Name:Drabek, Yan player_id list:['5172713']
Processing player_id:5172713
dir exists: file count:169 dir:e:\bridge\data\acbl\players\5172713\tournaments
dir not empty -- skipping
11:24:41 Name:Schmahl, Michael player_id list:['4934148']
Processing player_id:4934148
dir exists: file count:114 dir:e:\bridge\data\acbl\players\4934148\tournaments
dir not empty -- skipping
11:24:41 Name:Kopper, David player_id list:['2212501']
Processing player_id:2212501
dir exists: file count:67 dir:e:\bridge\data\acbl\players\2212501\tournaments
dir not empty -- skipping
11:24:41 Name:Denny, Christine player_id list:['32251

File count correct: e:\bridge\data\acbl\players\5546184\tournaments: terminating 5546184 early.
Processing player_id:9650423
dir exists: file count:216 dir:e:\bridge\data\acbl\players\9650423\tournaments
dir not empty -- skipping
Processing player_id:2258064
dir exists: file count:0 dir:e:\bridge\data\acbl\players\2258064\tournaments
41/245 gets:5 rate:0.79 url:https://api.acbl.org/v1/tournament/player/history_query?acbl_number=2258064&page=1&page_size=50&start_date=1900-01-01
File count correct: e:\bridge\data\acbl\players\2258064\tournaments: terminating 2258064 early.
Processing player_id:7428480
dir exists: file count:0 dir:e:\bridge\data\acbl\players\7428480\tournaments
41/245 gets:6 rate:0.82 url:https://api.acbl.org/v1/tournament/player/history_query?acbl_number=7428480&page=1&page_size=50&start_date=1900-01-01
File count correct: e:\bridge\data\acbl\players\7428480\tournaments: terminating 7428480 early.
Processing player_id:5074568
dir exists: file count:0 dir:e:\bridge\data\a

File count correct: e:\bridge\data\acbl\players\8987106\tournaments: terminating 8987106 early.
Processing player_id:4258207
dir exists: file count:314 dir:e:\bridge\data\acbl\players\4258207\tournaments
dir not empty -- skipping
11:24:52 Name:French, Andrea player_id list:['8783578']
Processing player_id:8783578
dir exists: file count:234 dir:e:\bridge\data\acbl\players\8783578\tournaments
dir not empty -- skipping
11:24:52 Name:Bishop, Dorn player_id list:['3412555']
Processing player_id:3412555
dir exists: file count:288 dir:e:\bridge\data\acbl\players\3412555\tournaments
dir not empty -- skipping
11:24:52 Name:Steele, Mark player_id list:['8479917']
Processing player_id:8479917
dir exists: file count:90 dir:e:\bridge\data\acbl\players\8479917\tournaments
dir not empty -- skipping
11:24:52 Name:Gaer, Jerry player_id list:['5034639']
Processing player_id:5034639
dir exists: file count:312 dir:e:\bridge\data\acbl\players\5034639\tournaments
dir not empty -- skipping
11:24:52 Name:Donn

...

File count correct: e:\bridge\data\acbl\players\9142649\tournaments: terminating 9142649 early.
11:27:09 Name:Yates, Tricia player_id list:['6862055']
Processing player_id:6862055
dir exists: file count:5 dir:e:\bridge\data\acbl\players\6862055\tournaments
dir not empty -- skipping
11:27:09 Name:Litovsky, Sam player_id list:['8849315']
Processing player_id:8849315
dir exists: file count:89 dir:e:\bridge\data\acbl\players\8849315\tournaments
dir not empty -- skipping
11:27:09 Name:Kimbrell, Corrine player_id list:['9675191']
Processing player_id:9675191
dir exists: file count:35 dir:e:\bridge\data\acbl\players\9675191\tournaments
dir not empty -- skipping
11:27:09 Name:Barbee, Carol player_id list:['6086438']
Processing player_id:6086438
dir exists: file count:46 dir:e:\bridge\data\acbl\players\6086438\tournaments
dir not empty -- skipping
11:27:09 Name:Howrey, Jim player_id list:['7262841']
Processing player_id:7262841
dir exists: file count:58 dir:e:\bridge\data\acbl\players\7262841\t

File count correct: e:\bridge\data\acbl\players\3226948\tournaments: terminating 3226948 early.
11:27:12 Name:Reuler, Ellen player_id list:['1356151']
Processing player_id:1356151
dir exists: file count:2 dir:e:\bridge\data\acbl\players\1356151\tournaments
dir not empty -- skipping
11:27:12 Name:Rappoport, Joel player_id list:['7525400']
Processing player_id:7525400
dir exists: file count:58 dir:e:\bridge\data\acbl\players\7525400\tournaments
dir not empty -- skipping
11:27:12 Name:Snyder, Angela player_id list:['5439906']
Processing player_id:5439906
dir exists: file count:230 dir:e:\bridge\data\acbl\players\5439906\tournaments
dir not empty -- skipping
11:27:12 Name:Miller, John player_id list:['7226098', '7739354', '5706459', '7738404', '4143485', '1422510', '5816793', '9198784', '9567437', '3928799']
Processing player_id:7226098
dir exists: file count:26 dir:e:\bridge\data\acbl\players\7226098\tournaments
dir not empty -- skipping
Processing player_id:7739354
dir exists: file count

500: count:134 skipping
Processing player_id:1584782
dir exists: file count:30 dir:e:\bridge\data\acbl\players\1584782\tournaments
dir not empty -- skipping
11:27:17 Name:Hastie, Chris player_id list:['3773663']
Processing player_id:3773663
dir exists: file count:174 dir:e:\bridge\data\acbl\players\3773663\tournaments
dir not empty -- skipping
11:27:17 Name:Stickan, Emery player_id list:['8761272']
Processing player_id:8761272
dir exists: file count:11 dir:e:\bridge\data\acbl\players\8761272\tournaments
dir not empty -- skipping
11:27:17 Name:Urquhart, Donald player_id list:['7574118', '4618912']
Processing player_id:7574118
dir exists: file count:0 dir:e:\bridge\data\acbl\players\7574118\tournaments
842/4944 gets:135 rate:0.98 url:https://api.acbl.org/v1/tournament/player/history_query?acbl_number=7574118&page=1&page_size=50&start_date=1900-01-01
File count correct: e:\bridge\data\acbl\players\7574118\tournaments: terminating 7574118 early.
Processing player_id:4618912
dir exists: fil

File count correct: e:\bridge\data\acbl\players\3579131\tournaments: terminating 3579131 early.
11:27:20 Name:Bay, Paul player_id list:['7396678']
Processing player_id:7396678
dir exists: file count:33 dir:e:\bridge\data\acbl\players\7396678\tournaments
dir not empty -- skipping
11:27:20 Name:Treanor, Bob player_id list:['9157972']
Processing player_id:9157972
dir exists: file count:19 dir:e:\bridge\data\acbl\players\9157972\tournaments
dir not empty -- skipping
11:27:20 Name:Butler, Dawn player_id list:['8309841']
Processing player_id:8309841
dir exists: file count:29 dir:e:\bridge\data\acbl\players\8309841\tournaments
dir not empty -- skipping
11:27:20 Name:McDougall, A Wylie player_id list:['5515033']
Processing player_id:5515033
dir exists: file count:80 dir:e:\bridge\data\acbl\players\5515033\tournaments
dir not empty -- skipping
11:27:20 Name:Silver, Maggie player_id list:['9401113']
Processing player_id:9401113
dir exists: file count:30 dir:e:\bridge\data\acbl\players\9401113\to

File count correct: e:\bridge\data\acbl\players\7763905\tournaments: terminating 7763905 early.
11:27:24 Name:Saunders, Ginny player_id list:['6811205']
Processing player_id:6811205
dir exists: file count:77 dir:e:\bridge\data\acbl\players\6811205\tournaments
dir not empty -- skipping
11:27:24 Name:Abeles, Michael player_id list:['7346654']
Processing player_id:7346654
dir exists: file count:78 dir:e:\bridge\data\acbl\players\7346654\tournaments
dir not empty -- skipping
11:27:24 Name:Waters, Amy Lou player_id list:['6646913']
Processing player_id:6646913
dir exists: file count:26 dir:e:\bridge\data\acbl\players\6646913\tournaments
dir not empty -- skipping
11:27:24 Name:Hill, Maria player_id list:['1343165']
Processing player_id:1343165
dir exists: file count:26 dir:e:\bridge\data\acbl\players\1343165\tournaments
dir not empty -- skipping
11:27:24 Name:Youssef, Youssef player_id list:['8055300']
Processing player_id:8055300
dir exists: file count:106 dir:e:\bridge\data\acbl\players\80